## Dealing with missing data - Homework 2

Author: Kriti Shrivastava (Spire ID: 31041848)

Sepetember 19th, 2017

Datasource: http://archive.ics.uci.edu/ml/datasets/Wholesale+customers

#### Description: 
Visualizing data set (with missing values) using Bokeh. Coding several approaches to deal with the missing data. Displaying the results with comparisons with the real values. 

###### Bokeh Version used: 0.12.6

    Importing necessary packages

In [50]:
from bokeh.plotting import figure, output_notebook, show
import pandas
import os
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import row
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS,BoxZoomTool, ResetTool, BoxSelectTool, LassoSelectTool
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Select
from collections import defaultdict
from bokeh.palettes import Spectral6

    Reading CSV data to Dataframes

In [51]:
path = os.getcwd()
original_path = path + '\data\Wholesale customers data.csv'
missing_path = path + '\data\Wholesale customers data-missing.csv'

original_data = pandas.read_csv(original_path)
missing_data = pandas.read_csv(missing_path)

output_notebook()

Loading BokehJS ...

   #### Technique 1: Ignore records with missing values
     
   #### Analysis:
   Since the number of missing records(6) is very less as compared to the total number of records(440 * 6 product types), the difference between the average spendings per product using the original data vs the average spendings per product for the data ignoring the missing records is negligible. There are only 1-2 missing values per product which do not impact the average calculation significantly. In such a use case, ignoring records with missing values can be an easy and effective technique to handle missing data.
   
   ##### Interactions: Linked Panning, Zoom, Reset

In [52]:
# Drop records with missing values
delete_missing_data = missing_data.dropna()

# Find number of records
no_of_records = len(original_data)
no_of_records_new = len(delete_missing_data) 

# Calculating average spendings using new data with deleted rows
total_values = {} 
total_values = defaultdict(lambda:0,total_values)
for record in delete_missing_data.itertuples():
    total_values['Fresh'] += record[3]
    total_values['Milk'] += record[4]
    total_values['Grocery'] += record[5]
    total_values['Frozen'] += record[6]
    total_values['Detergents_Paper'] += record[7]
    total_values['Delicassen'] += record[8]

# Calculating average spendings using original data
original_total_values = {}
original_total_values = defaultdict(lambda:0,original_total_values)
for record in original_data.itertuples():
    original_total_values['Fresh'] += record[3]
    original_total_values['Milk'] += record[4]
    original_total_values['Grocery'] += record[5]
    original_total_values['Frozen'] += record[6]
    original_total_values['Detergents_Paper'] += record[7]
    original_total_values['Delicassen'] += record[8]

types = list(total_values.keys())
counts = list(round(value/no_of_records_new,2) for value in total_values.values())
original_counts = list(round(value/no_of_records,2) for value in original_total_values.values())
source = ColumnDataSource(data=dict(types=types, counts=counts, original_counts=original_counts, color=Spectral6))

# Bar Chart 1: Original Data: Average Fresh/Milk/Grocery/Frozen/Detergents_Paper/Delicassen
p = figure(plot_width=520, plot_height=500, title="Original Data", x_range=types, y_range=(min(original_counts),max(original_counts)))
p.vbar(x='types', width=0.5, source = source, color='color', legend='types', top='original_counts')
p.xaxis.axis_label = 'Type of product'
p.yaxis.axis_label = 'Average spendings'
labels = LabelSet(x='types', y='original_counts', text='original_counts', level='glyph',
        x_offset=-15, y_offset=0, source=source, render_mode='canvas', text_font_size="10px")
p.add_layout(labels)

# Bar chart 2: Data with deleted records: Average Fresh/Milk/Grocery/Frozen/Detergents_Paper/Delicassen
p1 = figure(plot_width=520, plot_height=500, title="Data excluding missing records",x_range=p.x_range, y_range=p.y_range)
p1.vbar(x='types', width=0.5, source = source, color='color', legend='types', top='counts')
p1.xaxis.axis_label = 'Type of product'
p1.yaxis.axis_label = 'Average spendings'
labels1 = LabelSet(x='types', y='counts', text='counts', level='glyph',
        x_offset=-15, y_offset=0, source=source, render_mode='canvas', text_font_size="10px")
p1.add_layout(labels1)

show(row(p,p1))

   #### Technique 2: Use global constant
     
   #### Analysis:
  If the intension is to analyze individual records and see exactly how many records of a particular type are missing, then it makes sense to replace the missing value with a global constant(in this case, -9900). Different color(red) is used for such records which makes it easy to see if the records of the desired product type for a region are missing. This information can be very useful.
   
   ##### Interactions: Single selection widget (Custom JS), zoom, pan, reset

In [53]:
# Fill -9900 in place of missing records
use_constant_data = missing_data.fillna(-9900)

# Adding separate colors for separating original data with data with filled values
colors = list()
for record in use_constant_data.itertuples():
    if record[3] ==-9900 or record[4] == -9900 or record[5] == -9900 or record[6] == -9900 or record[7] == -9900 or record[8] == -9900 :
        colors.append("Red")
    else:
        colors.append("Blue")

product_types = ['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen']

use_constant_source = ColumnDataSource(use_constant_data)


scatterplot_data = {
    'Product': list(use_constant_data['Milk']),
    'Region': list(use_constant_data['Region'])
}    
scatterplot_source = ColumnDataSource(data=scatterplot_data)

callback =  CustomJS(args=dict(scatterplot_source=scatterplot_source, use_constant_source=use_constant_source), code="""
        var data = scatterplot_source.data;  
        var all_data = use_constant_source.data;
        var selected_product = cb_obj.value
        data['Product'] = all_data[selected_product]
        x = data['Region']
        y = data['Product']
        scatterplot_source.change.emit();
    """)

# Scatterplot
p = figure(plot_width=500, plot_height=500, tools="hover,pan,zoom_out,zoom_in,help,reset",title="Region VS spending on a product type")
p.circle( y='Product', x='Region', source=scatterplot_source, fill_color=colors, alpha=0.8, fill_alpha = 0.4, size=10)
p.xaxis.axis_label = 'Region'
p.yaxis.axis_label = 'Spending on a product type'

select = Select(title="Select Product type:", value="Milk", options=product_types, callback=callback)

layout = column(select, p)
show(layout)

C:\Users\Kriti Shrivastava\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


   #### Technique 3: Use atribute mean
     
   #### Analysis:
   In the graphs below, only the records with some missing data are shown. The missing data is filled with the atribute mean. Such records are shown in red and can be easily compared with their original values by selecting and the point and comparing the its value from the plot with original data. It can be seen that while for some records(ex 'Milk'), the mean is close to the original record, for most of the records, the atribute mean is very far off from its real value. 
   
   This technique would be suitable for the cases when some legitimate value is required for the missing data but the it is not necessary to get it exactly right. For example in a scenario when the focus of analysis is more on other atributes and the missing value does not have a significant effect on this analysis. 
   
   ##### Interactions: Linked brushing, Single selection widget (Custom JS), custom hover, reset

In [54]:
# Fill missing values with atribute mean
use_mean_data = missing_data.fillna(missing_data.mean())

# Selecting rows only with missing data
use_mean_data = use_mean_data.loc[use_mean_data['Missing'] == 'yes']
corresponding_real_data = original_data.loc[original_data['Missing'] == 'yes']

# Merging columns with original value
s1 = pandas.Series(list(corresponding_real_data.Milk), name='Milk_real')
s2 = pandas.Series(list(corresponding_real_data.Grocery), name='Grocery_real')
s3 = pandas.Series(list(corresponding_real_data.Detergents_Paper), name='Detergents_Paper_real')
s4 = pandas.Series(list(corresponding_real_data.Delicassen), name='Delicassen_real')

use_mean_data['Milk_real'] = list(s1)
use_mean_data['Grocery_real'] = list(s2)
use_mean_data['Detergents_Paper_real'] = list(s3)
use_mean_data['Delicassen_real'] = list(s4)

colors = []
for record in use_mean_data.itertuples():
    if record.Milk != record.Milk_real:
        colors.append("red")
    else:
        colors.append("blue")


mean_data = {
    'y1': list(use_mean_data['Milk']),
    'y2': list(use_mean_data['Milk_real']),
    'x': list(use_mean_data['Fresh']),
    'colors': colors
}    

mean_source = ColumnDataSource(data=mean_data)
all_mean_source = ColumnDataSource(use_mean_data)

callback =  CustomJS(args=dict(mean_source=mean_source, all_mean_source=all_mean_source), code="""
        var data = mean_source.data;  
        var all_data = all_mean_source.data;
        var selected_y = cb_obj.value;
        var selected_y2 = selected_y.concat("_real");
        data['y1'] = all_data[selected_y];
        data['y2'] = all_data[selected_y2];
        var colors = new Array()
        for(i=0;i<6;i++){
            if(data['y1'][i]!=data['y2'][i]){
                colors[i]='red';
                data['y1'][i] = Math.round(data['y1'][i])
            }
            else
                colors[i]='blue';
        }
        data['colors']=colors;
        y1 = data['y1']
        y2 = data['y2']
        x = data['Fresh']
        colors = data['colors']
        console.log(mean_source)
        mean_source.change.emit();
    """)

options = ['Milk','Grocery','Detergents_Paper','Delicassen']
hover = HoverTool(tooltips=[
    ("y", "@y1")
])

hover1 = HoverTool(tooltips=[
    ("Original value", "@y2")
])

plot = figure(plot_width=490, plot_height=500,y_range=(-1000,10000), tools=[hover,BoxSelectTool(),LassoSelectTool(), ResetTool()], title="Missing data replaced with atribute mean")
plot.circle(x='x', y='y1', source=mean_source, alpha=0.8, size=10, fill_color='colors', fill_alpha=0.6)
plot.yaxis.axis_label = 'Spendings on Product Selected'
plot.xaxis.axis_label = 'Spending on Fresh products'

plot1 = figure(plot_width=490, plot_height=500,y_range=(-1000,10000), tools=[hover1,BoxSelectTool(),LassoSelectTool(), ResetTool()], title="Original Data")
plot1.circle(x='x', y='y2', source=mean_source, alpha=0.8, size=10, fill_color='colors', fill_alpha=0.6)
plot1.yaxis.axis_label = 'Spendings on Product Selected'
plot1.xaxis.axis_label = 'Spending on Fresh products'

select = Select(title="Select Product on Y axis:", value="Milk", options=options, callback=callback)

layout = column(select, row(plot, plot1))

show(layout)

 #### Technique 4: Use most probable value
     
   #### Analysis:
   For this part, linear interpolation is used to find the most probable value for the missing data. It predicts almost accurate values for product types 'Delicassen' and 'Detergent_paper' and not so well for 'Milk' and 'Grocery'. Different methods like quadratic, spline, pchip could be tried and to see what works best.
   
   This technique is would be suitable for the cases when the missing value has a significant impact on the analysis. In such cases, it would be crucial to estimate the missing data as accurately as possible rather than substituting it with a global constant or the atribute predicted. 
   
   ##### Interactions: Linked brushing, Single selection widget (Custom JS), custom hover, reset

In [55]:
# Predict missing values using Linear Interpolation
use_predicted_data = missing_data.interpolate()

# Selecting rows only with missing data
use_predicted_data = use_predicted_data.loc[use_predicted_data['Missing'] == 'yes']
corresponding_real_data = original_data.loc[original_data['Missing'] == 'yes']

# Merging columns with original value
s1 = pandas.Series(list(corresponding_real_data.Milk), name='Milk_real')
s2 = pandas.Series(list(corresponding_real_data.Grocery), name='Grocery_real')
s3 = pandas.Series(list(corresponding_real_data.Detergents_Paper), name='Detergents_Paper_real')
s4 = pandas.Series(list(corresponding_real_data.Delicassen), name='Delicassen_real')

use_predicted_data['Milk_real'] = list(s1)
use_predicted_data['Grocery_real'] = list(s2)
use_predicted_data['Detergents_Paper_real'] = list(s3)
use_predicted_data['Delicassen_real'] = list(s4)

colors = []
for record in use_predicted_data.itertuples():
    if record.Milk != record.Milk_real:
        colors.append("red")
    else:
        colors.append("blue")


predicted_data = {
    'y1': list(use_predicted_data['Milk']),
    'y2': list(use_predicted_data['Milk_real']),
    'x': list(use_predicted_data['Fresh']),
    'colors': colors
}    

predicted_source = ColumnDataSource(data=predicted_data)
all_predicted_source = ColumnDataSource(use_predicted_data)

callback =  CustomJS(args=dict(predicted_source=predicted_source, all_predicted_source=all_predicted_source), code="""
        var data = predicted_source.data;  
        var all_data = all_predicted_source.data;
        var selected_y = cb_obj.value;
        var selected_y2 = selected_y.concat("_real");
        data['y1'] = all_data[selected_y];
        data['y2'] = all_data[selected_y2];
        var colors = new Array()
        for(i=0;i<6;i++){
            if(data['y1'][i]!=data['y2'][i])
                colors[i]='red';
            else
                colors[i]='blue';
        }
        data['colors']=colors;
        y1 = data['y1']
        y2 = data['y2']
        x = data['Fresh']
        colors = data['colors']
        console.log(predicted_source)
        predicted_source.change.emit();
    """)

options = ['Milk','Grocery','Detergents_Paper','Delicassen']

hover = HoverTool(tooltips=[
    ("y", "@y1")
])

hover1 = HoverTool(tooltips=[
    ("Original value", "@y2")
])

plot = figure(plot_width=490, plot_height=500, y_range=(-1000,21000), tools=[hover,BoxSelectTool(),LassoSelectTool(), ResetTool()], title="Value predicted using Linear Interpolation")
plot.circle(x='x', y='y1', source=predicted_source, alpha=0.8, size=10, fill_color='colors', fill_alpha=0.6)
plot.yaxis.axis_label = 'Spendings on Product Selected'
plot.xaxis.axis_label = 'Spending on Fresh products'

plot1 = figure(plot_width=490, plot_height=500,y_range=(-1000,21000), tools=[hover1,BoxSelectTool(),LassoSelectTool(), ResetTool()], title="Original Data")
plot1.circle(x='x', y='y2', source=predicted_source, alpha=0.8, size=10, fill_color='colors', fill_alpha=0.6)
plot1.yaxis.axis_label = 'Spendings on Product Selected'
plot1.xaxis.axis_label = 'Spending on Fresh products'

select = Select(title="Select Product on Y axis:", value="Milk", options=options, callback=callback)

layout = column(select, row(plot, plot1))

show(layout)